In [1]:
import json
import requests

In [2]:
country_code = "FR"
mailto = "bso@recherche.gouv.fr"
per_page = "200" # up to 200
openalex_url = f"https://api.openalex.org/institutions?filter=country_code:{country_code}&select=display_name,display_name_acronyms,id&per_page={per_page}&mailto={mailto}"

In [3]:
def get_count_from_openalex():
    response = requests.get(url=openalex_url)
    return response.json().get("meta", {}).get("count", 0)

In [6]:
def get_institutions_names_from_openalex(cursor):
    names = {}
    response = requests.get(url=f"{openalex_url}&cursor={cursor}")
    next_cursor = response.json().get("meta", {}).get("next_cursor")
    for result in response.json().get("results"):
        acronym = result.get("display_name_acronyms", [])[0] if len(result.get("display_name_acronyms", [])) > 0 else result.get("display_name")
        names[result.get("id").replace("https://openalex.org/", "").lower()] = {
            "commentsName": f"de {acronym}",
            "commentsNameEN": f"of {acronym}",
            "name": result.get("display_name")
        }
    return { "names": names, "next_cursor": next_cursor }

In [7]:
count = get_count_from_openalex()
openalex = {}
cursor = "*"
while len(list(openalex.keys())) < count:
    response = get_institutions_names_from_openalex(cursor)
    cursor = response.get("next_cursor")
    openalex = { **openalex, **response.get("names") }

In [8]:
with open("openalex.json", "w") as f:
    json.dump(openalex, f, indent=4)